## Data

In [12]:
import torch
import torch.nn as nn
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 14
sequence_length = 7

In [13]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>", "<sos>", "<eos>"]
                                 )
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'ăn': 13,
 'nên': 9,
 'nhớ': 8,
 'trồng': 12,
 '<eos>': 3,
 'kẻ': 7,
 'thì': 11,
 'có': 6,
 'quả': 10,
 '<sos>': 2,
 'chí': 4,
 '<unk>': 0,
 'cây': 5,
 '<pad>': 1}

In [14]:
data_x = []
data_y = []

corpus[0] = '<sos> ' + corpus[0] + ' <eos>'
corpus[1] = '<sos> ' + corpus[1] + ' <eos>'

for vector in corpus:
    vector = vector.split()
    data_x.append(vector[:-1])
    data_y.append(vector[1:])  

print(data_x)
print(data_y)

[['<sos>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây'], ['<sos>', 'có', 'chí', 'thì', 'nên']]
[['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', '<eos>'], ['có', 'chí', 'thì', 'nên', '<eos>']]


In [15]:
# Tokenize and numericalize your samples
def vectorize(x, y, vocab, sequence_length):    
    x_ids = [vocab[token] for token in x][:sequence_length]
    x_ids = x_ids + [vocab["<pad>"]] * (sequence_length - len(x))

    y_ids = [vocab[token] for token in y][:sequence_length]
    y_ids = y_ids + [vocab["<pad>"]] * (sequence_length - len(y))
    
    return x_ids, y_ids

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, vocab, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

In [16]:
for x, y in zip(data_x_ids, data_y_ids):
    print(x)
    print(y)
    print()

[2, 13, 10, 8, 7, 12, 5]
[13, 10, 8, 7, 12, 5, 3]

[2, 6, 4, 11, 9, 1, 1]
[6, 4, 11, 9, 3, 1, 1]



In [17]:
data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
print(data_x_ids.shape)

torch.Size([2, 7])


In [18]:
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)
print(data_y_ids.shape)

torch.Size([2, 7])


## Train with full data

In [19]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim)
        self.mask = torch.triu(torch.ones(sequence_length, sequence_length), 
                               diagonal=1).bool()
        self.transformer = nn.TransformerEncoderLayer(d_model=embed_dim, 
                                                       nhead=num_heads, 
                                                       batch_first=True, 
                                                       dim_feedforward=4,
                                                       dropout=0.0)
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)                        # [n, seq_len, embed_dim]
        x = self.transformer(x, src_mask=self.mask)  # [n, seq_len, embed_dim]        
        x = self.linear(x)                           # [n, seq_len, vocab_size]
        return x.permute(0,2,1)                      # [n, vocab_size, seq_len]

model = TG_Model(vocab_size, 8, 1, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(14, 8)
  (transformer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=False)
    )
    (linear1): Linear(in_features=8, out_features=4, bias=False)
    (dropout): Dropout(p=0.0, inplace=False)
    (linear2): Linear(in_features=4, out_features=8, bias=False)
    (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.0, inplace=False)
    (dropout2): Dropout(p=0.0, inplace=False)
  )
  (linear): Linear(in_features=8, out_features=14, bias=True)
)


In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [24]:
# train
for _ in range(40):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.858381509780884
2.284752368927002
1.8958898782730103
1.5912725925445557
1.3103694915771484
1.0587754249572754
0.8394702076911926
0.6621471643447876
0.5165800452232361
0.39091357588768005
0.2921273410320282
0.22718408703804016
0.1841728687286377
0.15798239409923553
0.13973678648471832
0.1269313544034958
0.11922446638345718
0.1151375100016594
0.11208752542734146
0.10876099020242691
0.10579776763916016
0.104253850877285
0.10377652943134308
0.10308627039194107
0.10199626535177231
0.10143332928419113
0.10146500170230865
0.10126148164272308
0.10062988102436066
0.10023339092731476
0.10029653459787369
0.10024523735046387
0.09989099949598312
0.09972202032804489
0.09984252601861954
0.09981755167245865
0.09959321469068527
0.09953796863555908
0.09964177757501602
0.09958101809024811


In [25]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=1))

tensor([[ 6, 10,  8,  7, 12,  5,  3],
        [ 6,  4, 11,  9,  3,  1,  1]])


In [26]:
data_y_ids

tensor([[13, 10,  8,  7, 12,  5,  3],
        [ 6,  4, 11,  9,  3,  1,  1]])

## Inference

In [27]:
promt = '<sos> có'
promt = promt.split()
promt_ids = [vocab[token] for token in promt][:sequence_length]
promt_ids = promt_ids + [vocab["<pad>"]] * (sequence_length - len(promt))

print(promt_ids)

[2, 6, 1, 1, 1, 1, 1]


In [28]:
for i in range(sequence_length - len(promt)):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    outputs = torch.argmax(outputs, axis=1)   
    next_id = outputs[0][len(promt)+i-1]

    promt_ids[len(promt)+i] = next_id.item()
    print(promt_ids)

[2, 6, 4, 1, 1, 1, 1]
[2, 6, 4, 11, 1, 1, 1]
[2, 6, 4, 11, 9, 1, 1]
[2, 6, 4, 11, 9, 3, 1]
[2, 6, 4, 11, 9, 3, 4]


In [ ]:
{'ăn': 13,
 'nên': 9,
 'nhớ': 8,
 'trồng': 12,
 '<eos>': 3,
 'kẻ': 7,
 'thì': 11,
 'có': 6,
 'quả': 10,
 '<sos>': 2,
 'chí': 4,
 '<unk>': 0,
 'cây': 5,
 '<pad>': 1}